In [1]:
import pandas as pd
import numpy as np
import math
import os

In [2]:
testdata_dir = "E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Federated_Differential_Methylation_Analysis\\Fed_test"

## Dasen normalisation results evaluation  
Comparing the betas calculated based on the dasen normalised methylated and unmethylated values obtained from the original R code (dasen() in the wateRmelon package), the centrally implemented pythonic translation of the dasen normalisation and the federated implementation of the pythonic translation of the dasen normalisation

In [3]:
R_dasen = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Data\\Data_Full_Datasets\\GSE66351\\Normalised_Betas.csv" , index_col=0)
R_dasen1 = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Federated_Differential_Methylation_Analysis\\Output\\QC_GSE66351\\Preprocessed_betas.csv", index_col=0)
R_dasen_test = R_dasen1.iloc[0:21, :]
Python_dasen = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Federated_Differential_Methylation_Analysis\\Output\\all_probes_samples_normalised_betas.csv", index_col=0)
Federated_dasen = None

In [4]:
# absolute difference between the beta values
if np.all(R_dasen.index == Python_dasen.index):
    dasen_diff_python = pd.Series.abs((R_dasen) - Python_dasen) # the (* -1) negates the sign difference between python and r calculated
    #regression coefficients
else: print("different probes in the two dataframes")

average_absolute_diff_python_dasen = dasen_diff_python.mean(axis = 1)
print(average_absolute_diff_python_dasen.mean())

0.00012390597584849053


In [33]:
print(dasen_diff_python.min(axis = 1))
print(dasen_diff_python.max(axis = 1))

cg00000029        3.115778e-07
cg00000108        1.486560e-05
cg00000109        9.469625e-06
cg00000165        4.344477e-07
cg00000236        1.359770e-05
                      ...     
ch.X.97129969R    7.627761e-07
ch.X.97133160R    2.174946e-07
ch.X.97651759F    4.763674e-08
ch.X.97737721F    3.380399e-07
ch.X.98007042R    4.918403e-08
Length: 450837, dtype: float64
cg00000029        0.000109
cg00000108        0.000139
cg00000109        0.000139
cg00000165        0.000117
cg00000236        0.000178
                    ...   
ch.X.97129969R    0.000214
ch.X.97133160R    0.000085
ch.X.97651759F    0.000058
ch.X.97737721F    0.000033
ch.X.98007042R    0.000143
Length: 450837, dtype: float64


In [34]:
absolute_average_python = Python_dasen.mean(axis=1)
absolute_average_R = R_dasen.mean(axis=1)

In [40]:
np.mean(((R_dasen - Python_dasen)/R_dasen).max(axis = 1))

0.0007340976231107149

In [31]:
absolute_av_diff = pd.Series.abs(absolute_average_R - absolute_average_python)
print(absolute_av_diff.max())
print(absolute_av_diff.min())
Python_dasen.iloc[:,np.argmax(pd.Series.abs(R_dasen.iloc[np.argmax(absolute_av_diff), :]) - pd.Series.abs(Python_dasen.iloc[np.argmax(absolute_av_diff), :]))]


0.002988128437193094
2.853828284798965e-12


cg00000029        0.506374
cg00000108        0.903468
cg00000109        0.794115
cg00000165        0.263778
cg00000236        0.729786
                    ...   
ch.X.97129969R    0.351454
ch.X.97133160R    0.565364
ch.X.97651759F    0.579893
ch.X.97737721F    0.414573
ch.X.98007042R    0.503866
Name: GSM2808926_3998920130_R02C01, Length: 450837, dtype: float64

In [28]:
np.max(pd.Series.abs((R_dasen) - Python_dasen)/R_dasen, axis = 1)

cg00000029        0.000173
cg00000108        0.000159
cg00000109        0.000189
cg00000165        0.000444
cg00000236        0.000224
                    ...   
ch.X.97129969R    0.000791
ch.X.97133160R    0.000264
ch.X.97651759F    0.000321
ch.X.97737721F    0.000131
ch.X.98007042R    0.000624
Length: 450837, dtype: float64

In [32]:
# root mean square error between the pythonic/federated beta values and the r beta values (used as ground truth values)
np.sqrt(pd.DataFrame.abs((R_dasen - Python_dasen).sum(axis=1)/len(R_dasen))) #for each probe

cg00000029        0.000103
cg00000108        0.000148
cg00000109        0.000167
cg00000165        0.000147
cg00000236        0.000168
                    ...   
ch.X.97129969R    0.000174
ch.X.97133160R    0.000052
ch.X.97651759F    0.000070
ch.X.97737721F    0.000075
ch.X.98007042R    0.000109
Length: 450837, dtype: float64

## EWAS results evaluation

In [5]:

R_EWAS = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Federated_Differential_Methylation_Analysis\\EWAS_GSE66351\\Results_dataset_test.csv", index_col=0)
R_EWAS_test = R_EWAS.iloc[0:21,:]
Python_EWAS = pd.read_csv("E:\\Msc Systems Biology\\MSB5000_Master_Thesis\\Practical work\\Federated_Differential_Methylation_Analysis\\Output\\results_diagnosis_regression_test_python.csv", index_col=0)
Federated_EWAS = pd.read_csv(os.path.join(testdata_dir, "Results_EWASa.csv"), header=[0,1], index_col=0)
Federated_EWAS_test = Federated_EWAS.iloc[0:21,:]

In [6]:
# check if the same probes are included in the test subset
if np.all(R_EWAS_test.index == Python_EWAS.index):
    coef_diff_python = pd.Series.abs((-1* R_EWAS_test["Diagnosis_Beta"]) - Python_EWAS["Diagnosis_Coef"]) # the (* -1) negates the sign difference between python and r calculated
    #regression coefficients
else: print("different probes in the two dataframes")

average_absolute_diff_python = coef_diff_python.mean()

if np.all(R_EWAS_test.index == Federated_EWAS_test.index):
    coef_diff_federated = pd.Series.abs((-1* R_EWAS_test["Diagnosis_Beta"]) - Federated_EWAS_test[("Coefficient", "Diagnosis")]) # the (* -1) negates the sign difference between python and r calculated
    #regression coefficients
else: print("different probes in the two dataframes")

average_absolute_diff_federated = coef_diff_federated.mean()

print(average_absolute_diff_python, average_absolute_diff_federated)

0.0014490096474050697 0.05228027048083186


In [23]:
pd.Series.abs(sum(((R_EWAS_test["Diagnosis_Beta"]*-1) - Federated_EWAS_test[("Coefficient", "Diagnosis")]))/len(R_EWAS_test["Diagnosis_Beta"]))

0.0241536102793927

In [7]:
#check root mean square error between the r (ground truth) and python generated regression coefficients
RMSE_python = math.sqrt(pd.Series.abs(sum(((R_EWAS_test["Diagnosis_Beta"] * -1) - Python_EWAS["Diagnosis_Coef"]))/len(R_EWAS_test["Diagnosis_Beta"])))
RMSE_federated = math.sqrt(pd.Series.abs(sum(((R_EWAS_test["Diagnosis_Beta"]*-1) - Federated_EWAS_test[("Coefficient", "Diagnosis")]))/len(R_EWAS_test["Diagnosis_Beta"])))
print(RMSE_python, RMSE_federated)

0.006869449574458739 0.1554143181286483


## Federated results evaluation  
Test how well it performs with imbalanced splits:  
* Sample size imbalance  
* Class lable imbalance

In [ ]:
# load in the test data